In [1]:
#libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import time
import folium
import os
import sys
sys.path.append(os.path.abspath('abstract_flows'))
import arrow
import grid
import flows

In [1]:
#%%script false --no-raise-error
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150
%config InLineBackend.figure_format = 'retina'
print('Retina screen activated')

Retina screen activated


In [2]:
#dir = '/home/user/Desktop/Datos/'
dir = '/Users/antoniomendez/Desktop/Tesis/Datos/datos_limpios/'

In [5]:
#estaciones_eco = np.load(dir + 'Adj_eco/matrices_estaciones/est_2019.npy')
estaciones_eco = np.load('/Users/antoniomendez/Desktop/Tesis/Datos/Adj_eco/matrices_estaciones/est_2019.npy')

In [6]:
data_eco = pd.read_csv(dir + 'ecobici/ecobici_2019.csv')
data_eco.head()

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,Tiempo_viaje
0,M,36,11444,67,2019-01-01,0:08:44,36,2019-01-01,0:14:03,5.316667
1,M,36,9196,55,2019-01-01,0:12:05,136,2019-01-01,0:42:11,30.100000
2,M,38,2612,29,2019-01-01,0:13:15,115,2019-01-01,0:37:47,24.533333
3,M,43,7463,29,2019-01-01,0:13:36,115,2019-01-01,0:38:04,24.466667
4,M,37,3987,136,2019-01-01,0:15:30,35,2019-01-01,0:22:47,7.283333


In [7]:
#%%script false --no-raise-error
# we only conserve the data of the third week of the year
data_eco = data_eco[(data_eco['Fecha_Retiro'] >= '2019-01-14') & (data_eco['Fecha_Retiro'] <= '2019-01-20')]
data_eco.head()

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,Tiempo_viaje
241974,M,35,2385,182,2019-01-14,0:00:39,75,2019-01-14,0:05:09,4.500000
241975,M,26,8840,425,2019-01-14,0:04:30,398,2019-01-14,0:07:10,2.666667
241976,M,35,12437,15,2019-01-14,0:04:31,12,2019-01-14,0:13:26,8.916667
241977,M,40,15082,360,2019-01-14,0:04:58,190,2019-01-14,0:32:48,27.833333
241978,M,55,10289,305,2019-01-14,0:06:36,30,2019-01-14,0:24:34,17.966667


In [8]:
data_eco.tail()

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,Tiempo_viaje
439008,F,30,11087,405,2019-01-20,23:56:17,182,2019-01-21,0:16:30,20.216667
439009,M,24,7238,372,2019-01-20,23:56:42,316,2019-01-21,0:04:45,8.050000
439010,M,19,11537,167,2019-01-20,23:56:43,371,2019-01-21,0:27:29,30.766667
439011,M,27,9700,295,2019-01-20,23:57:22,299,2019-01-21,0:01:47,4.416667
439012,M,24,12052,211,2019-01-20,23:59:03,452,2019-01-21,0:05:20,6.283333


In [9]:
hours = [f'{i}:00:00' for i in range(1, 24)] + ['0:00:00']

In [10]:
dates = data_eco['Fecha_Retiro'].unique()
dates = [date for date in dates if date >= '2019-01-01' and date <= '2019-12-31']
dates.sort()

In [11]:
def create_adjacency_matrix(flows_df, dims, directed=True):
    '''
    Parameters:
        flows_df (DataFrame): DataFrame with columns i_A, j_A, i_B, j_B, flow_count.
        dims (tuple): Dimensions of the grid (rows, columns).
        directed (bool): If True, the graph is directed; if False, the graph is undirected.

    Returns:
        adjacency_matrix (numpy.ndarray): Adjacency matrix of the graph with size (dims[0]*dims[1], (dims[0]*dims[1])).
        nodes (list): List of nodes (i, j) in the same order as the adjacency matrix.
    '''
    # Initialize the adjacency matrix with zeros
    n = dims[0] * dims[1]
    adjacency_matrix = np.zeros((n, n), dtype=int)
    
    # Create a mapping from (i, j) to a unique index
    def node_to_index(i, j):
        return i * dims[1] + j
    
    # Fill the adjacency matrix
    for _, row in flows_df.iterrows():
        i_A, j_A = row['i_A'], row['j_A']
        i_B, j_B = row['i_B'], row['j_B']
        flow_count = row['flow_count']
        
        # Convert (i, j) to indices
        idx_A = node_to_index(i_A, j_A)
        idx_B = node_to_index(i_B, j_B)
        
        # Update the adjacency matrix
        adjacency_matrix[idx_A, idx_B] += flow_count
        
        if not directed:
            adjacency_matrix[idx_B, idx_A] += flow_count
    
    # Generate the list of nodes in the same order as the adjacency matrix
    nodes = [(i, j) for i in range(dims[0]) for j in range(dims[1])]
    
    return adjacency_matrix, nodes

In [12]:
matrices = []
nodes = []
laplacians = []
dates_considered = []

In [13]:
# it will take like a minute per date
for date in dates:
    print('Processing: ', date)
    for hour in hours:
        if hour == '0:00:00':
            current_data = data_eco[(data_eco['Fecha_Retiro'] == date) & (data_eco['Hora_Retiro'] >= '23:00:00')]
        else:
            current_data = data_eco[(data_eco['Fecha_Retiro'] == date) & (data_eco['Hora_Retiro'] <= hour)]
        if current_data.empty:
            continue
        current_counter = flows.count_trips_ecobici(current_data, threshold=1, directed=True)
        current_stations = pd.unique(np.concatenate((current_counter['Est_A'].unique(), current_counter['Est_B'].unique())))
        current_grid = grid.Grid(5, 3, 'ecobici')
        current_station_cells = flows.stations_and_cells(current_grid.geodataframe(), current_stations, estaciones_eco)
        current_graph_df = flows.abstract_flows(current_counter, current_grid.geodataframe(), current_station_cells, estaciones_eco, threshold=1)
        current_matrix, current_nodes = create_adjacency_matrix(current_graph_df, [5, 3], directed=True)
        matrices.append(current_matrix)
        nodes.append(current_nodes)
        date_ = date + ' ' + hour
        date_ = datetime.strptime(date_, '%Y-%m-%d %H:%M:%S')
        dates_considered.append(int(time.mktime(date_.timetuple())))

Processing:  2019-01-14
Processing:  2019-01-15
Processing:  2019-01-16
Processing:  2019-01-17
Processing:  2019-01-18
Processing:  2019-01-19
Processing:  2019-01-20


In [14]:
np.save('vector_files/adjacency_matrices.npy', matrices)
np.save('vector_files/nodes.npy', nodes)
np.save('vector_files/dates_considered.npy', dates_considered)

In [2]:
days_to_consider = 5
total_train = 24 * days_to_consider

matrices = np.load('vector_files/adjacency_matrices.npy')
nodes = np.load('vector_files/nodes.npy')
dates_considered = np.load('vector_files/dates_considered.npy')

train_data = matrices[:total_train]
train_nodes = nodes[:total_train]
train_dates = dates_considered[:total_train]

test_data = matrices[total_train:]
test_nodes = nodes[total_train:]
test_dates = dates_considered[total_train:]

np.save('vector_files/train_data.npy', train_data)
np.save('vector_files/train_nodes.npy', train_nodes)
np.save('vector_files/train_dates.npy', train_dates)   
np.save('vector_files/test_data.npy', test_data)
np.save('vector_files/test_nodes.npy', test_nodes)
np.save('vector_files/test_dates.npy', test_dates)

In [3]:
def getcompmatrix():
	holiday = [i for i in range(1506765600, 1507478400+3600, 3600)]
	weekd = [1,2,3,4,5]
	noweekd = [6,0]
	listtime = train_dates
	
	
	co = np.zeros((listtime.shape[0],listtime.shape[0]))
	#print(co.shape)
	for i in range(co.shape[0]):
		tod = int(((listtime[i]-1427644800)/3600/24+1)%7)
		#print(tod)
		for j in range(co.shape[1]):
			tod2 = int(((listtime[j]-1427644800)/3600/24+1)%7)
			if (tod in weekd) & (tod2 in weekd) & (abs(i-j)%24==0) & (listtime[i] not in holiday) & (listtime[j] not in holiday):
				co[i][j] = 1
			if ((tod in noweekd) | (listtime[i] in holiday)) & ((tod2 in noweekd) | (listtime[j] in holiday)) & (abs(i-j)%24==0):
				co[i][j] = 1
	#for i in range(co.shape[0]):
		#print(co[i].sum())
	#print(co.sum())
	np.save("vector_files/cm_train.npy",co)
	
getcompmatrix()